In [207]:
import yfinance as yf 
from yfinance import EquityQuery
import numpy as np
import pandas as pd


In [208]:
# Defining a screen for US stocks in specific sectors with certain criteria -- presecreen for options volatility 
screen = EquityQuery("and",[EquityQuery("is-in", ["region", "us"]), EquityQuery("is-in", ["sector",
    "Technology",
    "Financial Services",
    "Consumer Cyclical",
    "Communication Services",
    "Basic Materials",
    "Industrials"
]), EquityQuery("GTE", ["eodprice",0.50])
,   EquityQuery("GTE", ["avgdailyvol3m", 100000]), 
EquityQuery("LTE", ["short_percentage_of_float.value", 100])])

screener = yf.screen(screen,sortField="avgdailyvol3m", sortAsc=False, size = 3)

stocks = screener.get('quotes', [])


In [209]:
stocks = pd.DataFrame(stocks)
stocks = stocks.drop(columns=[
    'esgPopulated',
    'cryptoTradeable',
    'hasPrePostMarketData',
    'firstTradeDateMilliseconds',
    'priceHint',
    'regularMarketDayHigh',
    'regularMarketDayRange',
    'regularMarketDayLow',
    'regularMarketPreviousClose',
    'messageBoardId',
    'fullExchangeName',
    'financialCurrency',
    'regularMarketOpen',
    'averageDailyVolume3Month',
    'averageDailyVolume10Day',
    'corporateActions',
    'fiftyTwoWeekLowChange',
    'fiftyTwoWeekLowChangePercent',
    'fiftyTwoWeekRange',
    'fiftyTwoWeekHighChange',
    'fiftyTwoWeekHighChangePercent',
    'fiftyTwoWeekChangePercent',
    'earningsTimestamp',
    'earningsTimestampStart',
    'earningsTimestampEnd',
    'earningsCallTimestampStart',
    'earningsCallTimestampEnd',
    'dividendRate',
    'trailingAnnualDividendYield',
    'marketState',
    'epsCurrentYear',
    'fiftyDayAverage',
    'fiftyDayAverageChange',
    'fiftyDayAverageChangePercent',
    'twoHundredDayAverage',
    'twoHundredDayAverageChange',
    'twoHundredDayAverageChangePercent','dividendYield','gmtOffSetMilliseconds','bid','ask','currency', 'quoteType','language','typeDisp','quoteSourceName','triggerable','customPriceAlertConfidence','sourceInterval','exchangeDataDelayedBy','exchangeTimezoneName','exchangeTimezoneShortName',
 'bidSize','askSize','market','isEarningsDateEstimate','trailingAnnualDividendRate','regularMarketTime','priceEpsCurrentYear','epsForward','forwardPE','tradeable','displayName','longName'
]
)
df = stocks

In [210]:
order = [
    # Identification
    'symbol',
    'shortName',
    'exchange',
    'region',
    
    # Market Price & Change
    'regularMarketPrice',
    'regularMarketChange',
    'regularMarketChangePercent',
    'regularMarketVolume',
    
    # Valuation Metrics
    'marketCap',
    'sharesOutstanding',
    'bookValue',
    'priceToBook',
    'trailingPE',
    'epsTrailingTwelveMonths',
    'averageAnalystRating',
    
    # Price Range
    'fiftyTwoWeekHigh',
    'fiftyTwoWeekLow',
    
    # Dividend Info
    'dividendDate'
]
df = df[order]
df


,symbol,shortName,exchange,region,regularMarketPrice,regularMarketChange,regularMarketChangePercent,regularMarketVolume,marketCap,sharesOutstanding,bookValue,priceToBook,trailingPE,epsTrailingTwelveMonths,averageAnalystRating,fiftyTwoWeekHigh,fiftyTwoWeekLow,dividendDate
0,NVDA,NVIDIA Corporation,NMS,US,139.2216,4.411606,3.272462,346437850,3395280699392,24387600384,34.304,4.058465,47.354286,2.94,1.5 - Strong Buy,153.13,86.62,1.743552e+09
1,F,Ford Motor Company,NYQ,US,10.1846,0.014600,0.143558,94597362,40499568640,3905700096,11.225,0.907314,8.147680,1.25,3.0 - Hold,14.85,8.44,1.748822e+09
2,TSLA,"Tesla, Inc.",NMS,US,358.6700,1.770020,0.495943,80032779,1155261792256,3220960000,23.184,15.470584,204.954300,1.75,2.6 - Hold,488.54,167.41,NaN


In [249]:
# Get the price history for the stocks, to do RSI, or MACD, options volatility, and kalman filter acceleration(or see tview workaround)
stocklist = df.symbol.tolist()
stocklist

# Get the price history for the stocks
price_data = yf.download(stocklist, period='14d', interval='1d')
price_data = price_data['Close']

# Calculate the daily returns
change_data = price_data[stocklist]/price_data[stocklist].shift(1)-1

# RSI
change_data
def rsi(price_data, period=14):
    delta = change_data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))
rsi_data = rsi(price_data)
rsi_data

rsi = round(rsi_data.iloc[-1],1)
df['RSI'] = rsi.values
#"""Steps:
#1. get options volatility, prices, data 
#2. get RSI, kalman filter acceleration
#indicators in screener: filter by RSI, filter by relative options volatility (vs historical volatility, as yfinance does not have historical options volatility data)
#"""

filterdf = df[df['RSI'] < 60]  # Filter for RSI < 60

## Rate of change 
window=5 
recent_avg = change_data.rolling(window=window).mean()
earlier_avg = change_data.shift(window).rolling(window=window).mean()

# Acceleration: difference between recent and earlier averages
accel = (recent_avg - earlier_avg) * 100
accel = round(accel.iloc[-1], 2)
df['ROC'] = accel.values


# Apply filters
filterdf = df[(df['RSI'] < 60) & (df['ROC'] > 1)] # Filter for RSI < 60

filterdf



[*********************100%***********************]  3 of 3 completed
/var/folders/f_/1bh6mfq97bxdwzwnr49jf8nm0000gn/T/ipykernel_33534/753487571.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['RSI'] = rsi.values


,symbol,shortName,exchange,region,regularMarketPrice,regularMarketChange,regularMarketChangePercent,regularMarketVolume,marketCap,sharesOutstanding,bookValue,priceToBook,trailingPE,epsTrailingTwelveMonths,averageAnalystRating,fiftyTwoWeekHigh,fiftyTwoWeekLow,dividendDate,RSI,ROC
0,NVDA,NVIDIA Corporation,NMS,US,139.2216,4.411606,3.272462,346437850,3395280699392,24387600384,34.304,4.058465,47.354286,2.94,1.5 - Strong Buy,153.13,86.62,1.743552e+09,45.8,1.64
2,TSLA,"Tesla, Inc.",NMS,US,358.6700,1.770020,0.495943,80032779,1155261792256,3220960000,23.184,15.470584,204.954300,1.75,2.6 - Hold,488.54,167.41,NaN,43.3,2.17
